# Machine Learning

## HSE, 2024-25

### Home Assignment #4. PCA

Assignment completed by:

    Ахременко Михаил

### General Information

__Publication date:__ 08.05.2024

__Deadline:__ 04:00 20.05.2024

### Grading and penalties

The number of points for each problem in this homework assignment is listed next to the problem condition.

The homework grade is calculated using the following formula:

$$
s \times 10/53 ,
$$

where $s$ is the number of points you have scored in total for all tasks.

For submitting an assignment after the deadline, a penalty of 1 **secondary** point per day is applied to the final grade for the assignment, but the delay cannot exceed one week.

__Attention!__ Homework assignments must be completed independently. "Similar" solutions are considered plagiarism, and all involved students (including those from whom the work was copied) will receive no more than 0 points for the assignment.

Additionally, please remember that all solutions are run through a special new anti-plagiarism system for Jupyter notebooks, which detects cross-similarities between different notebooks, as well as solutions generated by neural networks. Such work will also be strictly considered as plagiarism.

### Submission format

You upload your solution using the link provided in the telegram channel. You need to upload a file with the extension .ipynb (Python notebook)

### About the assignment

In this assignment, we will practice working with linear algebra and, specifically, with the PCA algorithm. The goal of the assignment is to try to compress the soundtrack of a Beethoven melody using this dimensionality reduction algorithm, which we have discussed in detail in seminars and lectures.

First, let's import all the necessary libraries:

In [ ]:
import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame
# Needed to read and write audio files
from scipy.io import wavfile

# This is to play audio files directly in Notebook
from IPython.display import Audio

# And this is the standard set of data analysis libraries required for this assignment
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

Let's download the data:

In [ ]:
# ! wget https://www.dropbox.com/s/p5147nr8mzemxnr/Beethoven_Violin_Sonata_Op_96_first_movement_bars_1-22.wav

Let's read the audio track using wavfile:


In [ ]:
samplerate, data = wavfile.read('Beethoven_Violin_Sonata_Op_96_first_movement_bars_1-22.wav')

In the cell above we see a certain `samplerate`. This variable stores the samplerate by default; the standard value for audio is 44100 Hz.

For those who are not very familiar with sound encoding and recording methods (aka those who didn't pass the 'EGE' in computer science at school :) ), - samplerate shows how many consecutive elements of the array with the signal encode a sound of 1 second duration.

You can learn more about sound encoding [here](https://ru.wikipedia.org/wiki/Кодирование_звуковой_информации).

So, let's see what the sampling rate of our audio track is.

In [ ]:
samplerate

If you divide the length of the signal array by `samplerate`, you will get the duration of the audio track in seconds. At least, you should get this result!

In [ ]:
len(data) / samplerate

Forty-five seconds. Does that sound about right? Compare it to the length of the track by running it on your computer.

Note also that the sound is stereo, as the signal is encoded with two channels (for the left and right speaker).

In [ ]:
data.shape

Draw the signals in both channels:

In [ ]:
# Channel 1
plt.figure(figsize=(20, 5))
plt.plot(data[:, 0])
plt.show()

# Channel 2
plt.figure(figsize=(20, 5))
plt.plot(data[:, 1])
plt.show()

Here it is - the legendary “cardiogram” sound we are all familiar with! :)

So now let's average the channels and get a mono sound, which will be much easier to work with.


In [ ]:
mono_sound = np.mean(data, axis=1)
mono_sound.shape

And now we're finally ready to hear what we have to compress! :)

In [ ]:
Audio(mono_sound, rate=samplerate)

Pretty, isn't it? :)

For convenience, we will also trim the signal array so that it can be easily divided into equal parts. These parts will form the dataset that you need to compress using the methods you know.

In fact, the method is very similar to the one we used to compress the picture at the seminar by splitting it into rectangular sub-pictures - only here the task is even simpler :)

In [ ]:
mono_sound_to_cut = mono_sound[:1990000]
# Audio(mono_sound_to_cut, rate=samplerate)

Let's check that our sound is now just a vector of numbers:

In [ ]:
mono_sound_to_cut.shape

Great, everything is absolutely ready!

Well, that's the end of our mission; now it's your turn! :)

### 1


#### 1.1. (2 points)
Write a function that will split the signal into equal parts of length 1000 and create a dataset represented as a two-dimensional array (a matrix of objects-features), where each part of the signal of length 1000 is in a separate row.

In [ ]:
def split_arr(arr: np.array, chunk_size: int = 1000, col_to_split: int = 0) -> np.array:
    if arr.ndim > 1:
        arr = arr[:, col_to_split]  # 0 - left column, 1 - the right one

    arr_len = len(arr)

    size_extend = 0
    if arr_len % chunk_size != 0:
        zero_to_add = chunk_size - arr_len % chunk_size
        arr = np.pad(
            array=arr,
            pad_width=(0, zero_to_add),
            mode='constant',
        )  # expand the array with zeros if it is not divisible by chunk_size
        size_extend = 1

    chunk_cnt = arr_len // chunk_size + size_extend
    arr = arr.reshape((chunk_cnt, chunk_size))

    return arr


CHUNK_SIZE = 1000

# left_signal_cut = get_split_arr(arr=data, chunk_size=CHUNK_SIZE, col_to_split=0)
mono_sound_split = split_arr(arr=mono_sound_to_cut, chunk_size=CHUNK_SIZE)
mono_sound_split
# print(len(mono_signal_cut))



#### 1.2. (3 points)

Write a function that translates your `matrix` back into an audio signal. In other words, the function unwraps data from a matrix of size `(number of objects, 1000)` into a vector of length `(number of objects * 1000)`.

Run the function and check that everything works correctly by playing the “restored” signal in your notebook - this signal should be exactly the same as the original one (because, in fact, it is).

In [ ]:
CHUNK_SIZE = 1000


def reshape_into_original(arr: np.array, chunk_size: int = 1000, original_size: int = None) -> np.array:
    original_size = original_size or len(arr)
    arr = arr[:original_size]
    arr = arr.reshape((len(arr) * chunk_size, ))

    return arr


def test_mono_signal_cut(arr1: np.array, arr2: np.array) -> None:
    assert np.array_equal(arr1, arr2)


mono_sound_reshaped_to_orig_after_split = reshape_into_original(mono_sound_split, CHUNK_SIZE)
print('Arrays are equal:', np.array_equal(mono_sound_reshaped_to_orig_after_split, mono_sound_to_cut))

Audio(mono_sound_reshaped_to_orig_after_split, rate=samplerate)


### 2





#### 2.1. (3 points)

Perform a PCA transformation of our matrix and get the data compressed into a lower dimensional space.

_Hint. At this stage, we have our "dataset" with 1000 "features" and we want to reduce the number of "features" using the PCA method. You are free to choose the number of components, but initially, it is advisable not to choose too small a number. This way, if the result is poor, it will be easier to determine whether the number of components was insufficient or if there was an error somewhere else :)_

    

In [ ]:
from sklearn.decomposition import PCA


N_COMPONENTS = {
    "best": 400,
    "nice": 300,
    "good": 200,
    "bad": 100,
    "very_bad": 50,
}
pca = PCA(n_components=N_COMPONENTS["good"])


mono_sound_compressed_w_pca = pca.fit_transform(mono_sound_split)
print("Compressed matrix size =", mono_sound_compressed_w_pca.shape, ' vs ', mono_sound_split.shape, 'as the initial one')

# check if we can decompress
mono_sound_decompressed_from_pca = pca.inverse_transform(mono_sound_compressed_w_pca)
mono_sound_decompressed_from_pca = reshape_into_original(mono_sound_decompressed_from_pca)

Audio(mono_sound_decompressed_from_pca, rate=samplerate)



#### 2.2. (4 points)

Visualize our "objects" in a clear form on a plane. Draw conclusions from their appearance.


После проекции аудиосигнала на двумерное пространство с помощью PCA наблюдается плотная центральная область, окружённая разреженными выбросами. Это свидетельствует о том, что большая часть аудиосегментов имеет схожие характеристики - менее амплитудный звук, тогда как некоторые участки (более амплитудные музыкальные события) существенно отклоняются от общего фона. Структура облака точек относительно симметрична, что говорит об отсутствии ярко выраженного тренда или направленного изменения структуры сигнала во времени.

_Hint. To do this, you need to apply PCA in some special way, which we also discussed in the seminar._

In [ ]:
N_COMPONENTS_2D = 2
pca_2d = PCA(n_components=N_COMPONENTS_2D)


mono_sound_compressed_w_pca_2d = pca_2d.fit_transform(mono_sound_split)
print(mono_sound_split.shape, ' vs ', mono_sound_compressed_w_pca_2d.shape)

plt.figure(figsize=(10, 10))

# ix = np.arange(len(mono_sound_compressed_w_pca_2d))
# plt.scatter(mono_sound_compressed_w_pca_2d[:, 0], mono_sound_compressed_w_pca_2d[:, 1], c=ix, cmap='jet')


plt.scatter(mono_sound_compressed_w_pca_2d[:, 0], mono_sound_compressed_w_pca_2d[:, 1], color="red")
plt.title("PCA sound projection to 2d plane")
plt.xlabel("first component")
plt.ylabel("second component")
plt.grid(True)
plt.show()


# plt.scatter(mono_sound_split[:, 0], mono_sound_split[:, 1], color="blue")
# plt.legend(["mono_sound_compressed_w_pca_2d", "mono_sound_split - two first columns"])
# plt.show()




In [ ]:
from pandas import DataFrame
a = DataFrame(mono_sound_compressed_w_pca_2d)
a.describe()

#### 2.3. (4 points)

Create a scatter plot of the dataset in space using color. Draw conclusions from this graph. What is depicted overall?


После проекции аудиосигнала на двумерное пространство с помощью PCA и цвета наблюдается большое количество точек в середине (цветом помечено именно количество точек). Это свидетельствует о том, что большая часть аудиосегментов имеет схожие характеристики - менее амплитудный звук, тогда как некоторые участки (более амплитудные музыкальные события) существенно отклоняются от общего фона. Структура облака точек относительно симметрична, что говорит об отсутствии ярко выраженного тренда или направленного изменения структуры сигнала во времени.



_Hint. That's what we did in the seminar too. This is another special application of PCA._

In [ ]:
pca_3d_or_2d = PCA(n_components=2)
mono_sound_compressed_w_pca_3d_or_2d = pca_3d_or_2d.fit_transform(mono_sound_split)

ix = np.arange(mono_sound_compressed_w_pca_3d_or_2d.shape[0])

plt.figure(figsize=(10, 10))

# ix = np.arange(len(mono_sound_compressed_w_pca_2d))
# plt.scatter(mono_sound_compressed_w_pca_2d[:, 0], mono_sound_compressed_w_pca_2d[:, 1], c=ix, cmap='jet')


plt.scatter(mono_sound_compressed_w_pca_3d_or_2d[:, 0], mono_sound_compressed_w_pca_3d_or_2d[:, 1], c=ix, cmap="PiYG")
plt.title("PCA sound projection to 2d plane")
plt.xlabel("first component")
plt.ylabel("second component")
plt.colorbar()
plt.grid(True)
# plt.show()


# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(3, 1, (1, 4), projection='3d')
#
# plot1 = ax.scatter(
#     mono_sound_compressed_w_pca_3d_or_2d[:, 0],
#     mono_sound_compressed_w_pca_3d_or_2d[:, 1],
#     mono_sound_compressed_w_pca_3d_or_2d[:, 2],
#     c=ix,
#     cmap='PiYG',
# )
#
# ax.set_title("3d")
# ax.set_xlabel("first component")
# ax.set_ylabel("second component")
# ax.set_zlabel("third component")
#
# fig.colorbar(plot1)
# plt.show()


### 3

We need to deal with the actual "compression" of the sound and verify the correctness of our actions.

#### 3.1. (4 points)

Perform inverse PCA transformation of the compressed data and get a “matrix” with the compressed sound. What will be the size of the resulting matrix?

Так я же вроде сделал это в 2.1. Ну ладно - повторим


In [ ]:
mono_sound_compressed_w_pca = pca.fit_transform(mono_sound_split)
print("Compressed matrix size =", mono_sound_compressed_w_pca.shape, ' vs ', mono_sound_split.shape, 'as the initial one')

# check if we can decompress
mono_sound_decompressed_from_pca = pca.inverse_transform(mono_sound_compressed_w_pca)
mono_sound_decompressed_from_pca = reshape_into_original(mono_sound_decompressed_from_pca)


#### 3.2. (4 points)

Convert the “matrix” obtained in the previous step into a playable signal (our “compressed” mono sound). Play the result `(Audio(YOUR_RESULT, rate = samplerate)`.

Does the resulting sound resemble the original?


Всё похоже - потому что взял хорошее качество, но плохую степень сжатия (1000 -> 200 фичей).
Но если кол-во фичей уменьшим до 30-ти, то качество будет очень плохое, но похожесть на оригинал всё равно будет заметна (нижи 30 - становится уже прям почти не похоже)


In [ ]:
N_COMPONENTS = {
    "best": 400,
    "nice": 300,
    "good": 200,
    "bad": 100,
    "very_bad": 50,
    "awful": 30,
}
pca = PCA(n_components=N_COMPONENTS["awful"])

mono_sound_compressed_w_pca = pca.fit_transform(mono_sound_split)
print("Compressed matrix size =", mono_sound_compressed_w_pca.shape, ' vs ', mono_sound_split.shape, 'as the initial one')

# check if we can decompress
mono_sound_decompressed_from_pca = pca.inverse_transform(mono_sound_compressed_w_pca)
mono_sound_decompressed_from_pca = reshape_into_original(mono_sound_decompressed_from_pca)


Audio(mono_sound_decompressed_from_pca, rate=samplerate)

#### 3.3. (8 points)

Conduct a study of the dependence of sound quality on the number of components.

Determine by ear the minimum number of components at which the sound is almost indistinguishable from the original.

Add two variants of the soundtrack to the cells: the original and the one you have chosen. Specify the number of components you have chosen.

_Hint. Try filtering the signal with the `gaussian_filter1d` function from `scipy.ndimage`. This will help remove the unpleasant shot noise when the compression is severe. You'll see what it's all about when you try it in practice._

_Sample code for filtering: `Audio(gaussian_filter1d(mono_sound_compressed, 2), rate = samplerate)`_


С gaussian_filter1d получается непохоже на оригинал - дажи при n_components=170. Хотя с другой стороны, на оригинал не похоже и с n_components=400 - звук будто чуть мутноватый, приглушённый.
Чисто на слух, после n_components=130 качество почти не меняется => можно сказать, что 130 - минимальное число компонент для того, чтобы звук был без явных дефектов


In [ ]:
from scipy.ndimage import gaussian_filter1d


N_COMPONENTS = {
    "best": 400,
    "nice": 300,
    "good": 200,
    "bad": 100,
    "very_bad": 50,
    "awful": 30,
}

def get_compressed_mono_sound(n_components: int, is_gaussian_filter1d_used: bool = True) -> np.array:
    ipca = PCA(n_components=n_components)
    mono_sound_compressed_w_pca1 = ipca.fit_transform(mono_sound_split)
    print("Compressed matrix size =", mono_sound_compressed_w_pca1.shape)
    mono_sound_decompressed_from_pca1 = ipca.inverse_transform(mono_sound_compressed_w_pca1)
    mono_sound_decompressed_from_pca1 = reshape_into_original(mono_sound_decompressed_from_pca1)

    if is_gaussian_filter1d_used:
        return Audio(gaussian_filter1d(mono_sound_decompressed_from_pca1, 2), rate=samplerate)
    return Audio(mono_sound_decompressed_from_pca1, rate=samplerate)


print("Original")
display(Audio(mono_sound, rate=samplerate))
# print("Best fake")
# display(get_compressed_mono_sound(N_COMPONENTS["best"], is_gaussian_filter1d_used=False))

for i in range(N_COMPONENTS["bad"], N_COMPONENTS["good"], 20):
    display(get_compressed_mono_sound(i))
    # display(get_compressed_mono_sound(i, is_gaussian_filter1d_used=False))








#### 3.4. (6 points)
Argumentatively answer the following questions:
- Is the number of components you have chosen too many or too few? How can you determine this?
- To what extent can the sound be compressed in this way? What memory optimization did you achieve in the process?
- If we are given a different soundtrack, we would need to do all the same things to compress the sound. How do we automatically select the number of components and is it possible?




1. 130 компонент - это 130 полей в матрице. Это много или мало? По мне так просто дофига! 1, 2, ну в крайнем случае 3 компонента - вот хорошее число.

2. Сократили кол-во столбцов в матрице примерно в 8 раз, что теоретически может уменьшить размер занимаемой ими памяти в 8 раз (практически же это не совсем ясно - wav может иметь свой алгоритм сжатия, жёсткий диск/ssd тоже может хранить данные по-разному, поэтому эта степень сжатия примерная).

3.


### 4

#### Extra Research. (15 points).

  - Wrap the resulting audio compression code in one or more functions.


см код ниже


  - Investigate how the compression ratio - the ratio of the size of the parts into which the signal was divided in task 1.1 to the size of the space into which you compressed the data using PCA - affects the sound, according to your subjective feelings. Starting from what compression level is the loss of audio track quality noticeable? (Both with and without `gaussian_filter1d` filtering).


Коэффициент сжатия — это отношение изначальной размерности (например, 1000) к числу компонент после PCA (например, 1000 / 130 ≈ 7.7x сжатие). Будем считать 8 раз
При сжатии выше чем 8 без фильтрации появляются шумы, искажения тембра, неприятные высокочастотные артефакты.
С фильтрацией gaussian_filter1d звук становится мягче ("должен становиться" - я бы не сказал, звук становится будто хуже), но проблемы аналогичные.
Субъективно: до 8x сжатия звук близок к оригиналу, после 8x — различия становятся заметны.


  - What does the compression ratio mean for PCA? For a large audio recording (3 min, for example), would we want to break it into more, less, or the same number of segments as the proposed audio recording? Why?


Для PCA коэффициент сжатия означает, насколько сильно уменьшается размерность входных данных (т.е. объём информации, необходимый для хранения и восстановления).

Если мы берём длинный трек (например, 3 минуты), нам стоит делить его на такое же или большее количество сегментов (то есть сегменты меньшей длины), потому что:
- аудиосигнал будет разнообразнее;
- разные части трека будут содержать разные частоты и динамику;
- лучше применять PCA отдельно к более коротким однородным отрезкам.
Меньшее число крупных сегментов ухудшит компрессию, так как статистика PCA "размоется" и станет менее точной.





  - Is there any way to automatically select the compression ratio? What is it responsible for in our task? How does the compression ratio affect the sound? Why does it affect the sound in this way?


Да, можно использовать:
- PCA(n_components=0.95): сохранит столько компонент, сколько нужно для покрытия 95% дисперсии;
- смотреть на график накопленной дисперсии ( cum_var = np.cumsum(pca.explained_variance_ratio_) ).
Коэффициент сжатия влияет на то, сколько информации мы сохраняем и сколько выбрасываем (в виде наименее важных компонент).
Он влияет на звук напрямую: меньше компонент -> меньше деталей, меньше тембровой глубины -> искажения и потери нюансов.
Это происходит потому, что PCA оставляет только наиболее "энергетически значимые" оси — а значит, мелкие детали, шумы, обертона — теряются при сильной компрессии.

In [ ]:
# 1.
def split_arr(arr: np.array, chunk_size: int = 1000, col_to_split: int = 0, n_components: int = 100) -> np.array:
    from sklearn.decomposition import PCA

    if arr.ndim > 1:
        arr = arr[:, col_to_split]  # 0 - left column, 1 - the right one

    arr_len = len(arr)

    size_extend = 0
    if arr_len % chunk_size != 0:
        zero_to_add = chunk_size - arr_len % chunk_size
        arr = np.pad(
            array=arr,
            pad_width=(0, zero_to_add),
            mode='constant',
        )  # expand the array with zeros if it is not divisible by chunk_size
        size_extend = 1

    chunk_cnt = arr_len // chunk_size + size_extend
    arr = arr.reshape((chunk_cnt, chunk_size))

    ipca = PCA(n_components=n_components)
    arr = ipca.fit_transform(arr)

    return arr
